# Constrained vs Unconstrained Portfolio Backtest

Goal:
- Compare your real 1k-style constrained execution against unconstrained signal execution.
- Quantify fill-rate drag and how constraints change realized expectancy.


In [ ]:
from __future__ import annotations

from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8-whitegrid")


def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "main.py").exists() and (p / "config.json").exists():
            return p
    return start

ROOT = find_repo_root(Path.cwd())
DATA = ROOT / "data"

CONSTRAINED_TRADES = DATA / "backtest_portfolio_trades_constrained_180d.csv"
CONSTRAINED_SKIPS = DATA / "backtest_portfolio_skips_constrained_180d.csv"
CONSTRAINED_SIGNALS = DATA / "backtest_portfolio_signals_constrained_180d.csv"

UNCONSTRAINED_TRADES = DATA / "backtest_portfolio_trades_unconstrained_180d.csv"
UNCONSTRAINED_SKIPS = DATA / "backtest_portfolio_skips_unconstrained_180d.csv"
UNCONSTRAINED_SIGNALS = DATA / "backtest_portfolio_signals_unconstrained_180d.csv"

for p in [
    CONSTRAINED_TRADES,
    CONSTRAINED_SKIPS,
    CONSTRAINED_SIGNALS,
    UNCONSTRAINED_TRADES,
    UNCONSTRAINED_SKIPS,
    UNCONSTRAINED_SIGNALS,
]:
    print(p, "exists=", p.exists())


In [ ]:
def load_frame(path: Path) -> pd.DataFrame:
    if not path.exists():
        return pd.DataFrame()
    try:
        df = pd.read_csv(path)
    except pd.errors.EmptyDataError:
        return pd.DataFrame()
    for col in ["entry_ts", "exit_ts", "signal_ts"]:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], utc=True, errors="coerce")
    return df

c_trades = load_frame(CONSTRAINED_TRADES)
c_skips = load_frame(CONSTRAINED_SKIPS)
c_signals = load_frame(CONSTRAINED_SIGNALS)

u_trades = load_frame(UNCONSTRAINED_TRADES)
u_skips = load_frame(UNCONSTRAINED_SKIPS)
u_signals = load_frame(UNCONSTRAINED_SIGNALS)

print("constrained trades", len(c_trades), "skips", len(c_skips), "signals", len(c_signals))
print("unconstrained trades", len(u_trades), "skips", len(u_skips), "signals", len(u_signals))


In [ ]:
def summarize(name: str, trades: pd.DataFrame, skips: pd.DataFrame, signals: pd.DataFrame) -> dict:
    total_signals = int(len(signals))
    executed = int(len(trades))
    skipped = int(len(skips))
    fill_rate = executed / total_signals if total_signals else np.nan
    win_rate = float((trades["r_multiple"] > 0).mean()) if executed else np.nan
    avg_r = float(trades["r_multiple"].mean()) if executed else np.nan
    med_r = float(trades["r_multiple"].median()) if executed else np.nan
    cum_r = float(trades["r_multiple"].sum()) if executed else 0.0
    return {
        "profile": name,
        "signals": total_signals,
        "executed": executed,
        "skipped": skipped,
        "fill_rate": fill_rate,
        "win_rate": win_rate,
        "avg_r": avg_r,
        "median_r": med_r,
        "cum_r": cum_r,
    }

summary = pd.DataFrame([
    summarize("constrained", c_trades, c_skips, c_signals),
    summarize("unconstrained", u_trades, u_skips, u_signals),
])
summary


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3.5))

summary.plot.bar(x="profile", y="fill_rate", ax=axes[0], legend=False, color=["#1f77b4", "#2ca02c"])
axes[0].set_title("Fill Rate")
axes[0].set_ylim(0, 1.05)

summary.plot.bar(x="profile", y="avg_r", ax=axes[1], legend=False, color=["#1f77b4", "#2ca02c"])
axes[1].set_title("Avg R")
axes[1].axhline(0, color="black", linewidth=1)

summary.plot.bar(x="profile", y="cum_r", ax=axes[2], legend=False, color=["#1f77b4", "#2ca02c"])
axes[2].set_title("Total R")
axes[2].axhline(0, color="black", linewidth=1)

plt.tight_layout()
plt.show()


In [ ]:
skip_counts = c_skips["skip_reason"].value_counts().rename_axis("reason").reset_index(name="count")
skip_counts


In [ ]:
if not skip_counts.empty:
    ax = skip_counts.plot.bar(x="reason", y="count", figsize=(6, 3), legend=False, color="#d62728")
    ax.set_title("Constrained Skip Reasons")
    plt.tight_layout()
    plt.show()


In [ ]:
# Realized sequence under each profile

def add_seq(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    out = df.sort_values("entry_ts").copy()
    out["trade_num"] = range(1, len(out) + 1)
    out["cum_r"] = out["r_multiple"].cumsum()
    return out

c_seq = add_seq(c_trades)
u_seq = add_seq(u_trades)

fig, ax = plt.subplots(figsize=(8, 4))
if not c_seq.empty:
    ax.plot(c_seq["trade_num"], c_seq["cum_r"], label="constrained", linewidth=2)
if not u_seq.empty:
    ax.plot(u_seq["trade_num"], u_seq["cum_r"], label="unconstrained", linewidth=2)
ax.set_title("Cumulative R Path")
ax.set_xlabel("Trade Number")
ax.set_ylabel("Cumulative R")
ax.axhline(0, color="black", linewidth=1)
ax.legend()
plt.tight_layout()
plt.show()


## Takeaway

- Yes, this absolutely makes sense to do.
- Unconstrained backtest answers: "Is the signal logic profitable if every signal is tradable?"
- Constrained backtest answers: "Is *my account* profitable given slots/capital/risk caps?"
- The gap between the two is your capacity tax.

Decision rule:
- Keep using unconstrained backtests to validate signal quality.
- Use constrained portfolio backtests as the final gate for deployability on a 1k account.


## Capacity sweep (slots x risk cap)

This section tests how increasing account capacity changes fill rate and expectancy for the same signal stream.


In [ ]:
sweep_rows = []
for slots in [1, 2, 3]:
    for risk in [25, 50]:
        trades_path = DATA / f"portfolio_sweep/s{slots}_r{risk}_trades.csv"
        skips_path = DATA / f"portfolio_sweep/s{slots}_r{risk}_skips.csv"
        signals_path = DATA / f"portfolio_sweep/s{slots}_r{risk}_signals.csv"

        t = load_frame(trades_path)
        k = load_frame(skips_path)
        s = load_frame(signals_path)

        total = int(len(s))
        executed = int(len(t))
        fill = executed / total if total else np.nan
        avg_r = float(t['r_multiple'].mean()) if executed else np.nan
        win = float((t['r_multiple'] > 0).mean()) if executed else np.nan
        cum_r = float(t['r_multiple'].sum()) if executed else 0.0

        sweep_rows.append({
            'slots': slots,
            'risk_cap': risk,
            'signals': total,
            'executed': executed,
            'fill_rate': fill,
            'win_rate': win,
            'avg_r': avg_r,
            'cum_r': cum_r,
            'skipped': int(len(k)),
        })

sweep = pd.DataFrame(sweep_rows).sort_values(['slots', 'risk_cap']).reset_index(drop=True)
sweep


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(13, 3.8))

pivot_fill = sweep.pivot(index='slots', columns='risk_cap', values='fill_rate')
pivot_avg = sweep.pivot(index='slots', columns='risk_cap', values='avg_r')
pivot_cum = sweep.pivot(index='slots', columns='risk_cap', values='cum_r')

im1 = axes[0].imshow(pivot_fill.values, cmap='Blues', aspect='auto', vmin=0, vmax=1)
axes[0].set_title('Fill Rate')
axes[0].set_xticks(range(len(pivot_fill.columns)))
axes[0].set_xticklabels(pivot_fill.columns)
axes[0].set_yticks(range(len(pivot_fill.index)))
axes[0].set_yticklabels(pivot_fill.index)
axes[0].set_xlabel('Risk cap (USD)')
axes[0].set_ylabel('Max open positions')
plt.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04)

im2 = axes[1].imshow(pivot_avg.values, cmap='RdYlGn', aspect='auto')
axes[1].set_title('Avg R')
axes[1].set_xticks(range(len(pivot_avg.columns)))
axes[1].set_xticklabels(pivot_avg.columns)
axes[1].set_yticks(range(len(pivot_avg.index)))
axes[1].set_yticklabels(pivot_avg.index)
axes[1].set_xlabel('Risk cap (USD)')
axes[1].set_ylabel('Max open positions')
plt.colorbar(im2, ax=axes[1], fraction=0.046, pad=0.04)

im3 = axes[2].imshow(pivot_cum.values, cmap='viridis', aspect='auto')
axes[2].set_title('Total R')
axes[2].set_xticks(range(len(pivot_cum.columns)))
axes[2].set_xticklabels(pivot_cum.columns)
axes[2].set_yticks(range(len(pivot_cum.index)))
axes[2].set_yticklabels(pivot_cum.index)
axes[2].set_xlabel('Risk cap (USD)')
axes[2].set_ylabel('Max open positions')
plt.colorbar(im3, ax=axes[2], fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()


In [ ]:
# Practical selection rule: maximize total R with a minimum avg R floor
candidates = sweep[sweep['avg_r'] >= 0.25].copy()
best = candidates.sort_values(['cum_r', 'fill_rate'], ascending=[False, False]).head(1)
best


Interpretation from this sweep:
- `slots` is the main lever in this dataset; increasing from 1 -> 3 sharply improves fill-rate and total R.
- `risk_cap` 25 vs 50 has little effect here (capital/slot limits dominate before risk cap binds).
- If you prioritize quality per trade, keep slots lower; if you prioritize monthly output, increase slots while monitoring avg R drift.


## Qty sweep (growth vs drawdown pressure)

Fixed profile for this sweep: `max_open_positions=3`, `max_capital_usd=1000`, `max_total_open_risk_usd=25`.


In [ ]:
def drawdown_stats(series: pd.Series) -> tuple[float, float]:
    if series.empty:
        return 0.0, 0.0
    running_max = series.cummax()
    dd = series - running_max
    max_dd = float(dd.min())
    max_dd_pct = float((dd / running_max.replace(0, np.nan)).min())
    if np.isnan(max_dd_pct):
        max_dd_pct = 0.0
    return max_dd, max_dd_pct

qty_rows = []
for qty_tag, qty in [("q0_25", 0.25), ("q0_5", 0.5), ("q1_0", 1.0)]:
    tpath = DATA / f"portfolio_sweep_qty/{qty_tag}_trades.csv"
    kpath = DATA / f"portfolio_sweep_qty/{qty_tag}_skips.csv"
    spath = DATA / f"portfolio_sweep_qty/{qty_tag}_signals.csv"

    t = load_frame(tpath).sort_values('entry_ts').copy()
    k = load_frame(kpath)
    s = load_frame(spath)

    if not t.empty:
        t['r_multiple'] = pd.to_numeric(t['r_multiple'], errors='coerce')
        t['risk_to_stop_usd'] = pd.to_numeric(t['risk_to_stop_usd'], errors='coerce')
        t['pnl_usd'] = t['r_multiple'] * t['risk_to_stop_usd']
        t['cum_r'] = t['r_multiple'].cumsum()
        t['cum_usd'] = t['pnl_usd'].cumsum()
        max_dd_r, _ = drawdown_stats(t['cum_r'])
        max_dd_usd, _ = drawdown_stats(t['cum_usd'])
    else:
        max_dd_r = 0.0
        max_dd_usd = 0.0

    qty_rows.append({
        'qty': qty,
        'signals': len(s),
        'executed': len(t),
        'skipped': len(k),
        'fill_rate': (len(t) / len(s)) if len(s) else np.nan,
        'avg_r': float(t['r_multiple'].mean()) if len(t) else np.nan,
        'cum_r': float(t['r_multiple'].sum()) if len(t) else 0.0,
        'cum_usd': float(t['pnl_usd'].sum()) if len(t) else 0.0,
        'max_dd_r': max_dd_r,
        'max_dd_usd': max_dd_usd,
    })

qty_summary = pd.DataFrame(qty_rows).sort_values('qty').reset_index(drop=True)
qty_summary


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3.5))

qty_summary.plot(x='qty', y='cum_usd', marker='o', ax=axes[0], legend=False)
axes[0].set_title('Total PnL (USD)')
axes[0].axhline(0, color='black', linewidth=1)

qty_summary.plot(x='qty', y='max_dd_usd', marker='o', ax=axes[1], legend=False, color='#d62728')
axes[1].set_title('Max Drawdown (USD)')
axes[1].axhline(0, color='black', linewidth=1)

qty_summary.plot(x='qty', y='avg_r', marker='o', ax=axes[2], legend=False, color='#2ca02c')
axes[2].set_title('Avg R (shape quality)')
axes[2].axhline(0, color='black', linewidth=1)

for ax in axes:
    ax.set_xlabel('qty')

plt.tight_layout()
plt.show()


Interpretation:
- `avg_r` typically stays similar across qty because it is risk-normalized.
- USD growth and USD drawdown should scale with qty.
- Use the highest qty that keeps max drawdown within your psychological and account-risk tolerance.
